<a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/intro/Report_API_Quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{python-report-api} -->

<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />
<!--- @wandbcode{python-report-api} -->

## What is the Report API?
- Programmatically create and modify reports in Python, including support for editing blocks, panels, and runsets.
- Create report templates to reuse and share with others



## Quick Links
- [🚀 Quickstart Guide](#quickstart) (~5min)
- [❓ FAQ](#faq)
- [📌 Complete Examples](#complete_examples)

In [61]:
!pip install wandb -qqq

## Setup

In [ ]:
#@title ## Log Runs { run: "auto", display-mode: "form" }
#@markdown If this is your first time here, consider running the setup code for a better docs experience!
#@markdown If you have run the setup code before, you can uncheck the box below to avoid unnecessary logging.

LOG_DUMMY_RUNS = True #@param {type: "boolean"}


import requests
from PIL import Image
from io import BytesIO
import wandb
import pandas as pd
from itertools import product
import random
import math

import wandb
import random
import string

ENTITY = wandb.apis.PublicApi().default_entity
PROJECT = "report-api-quickstart" #@param {type: "string"}
LINEAGE_PROJECT = "lineage-example" #@param {type: "string"}


def get_image(url):
    r = requests.get(url)
    return Image.open(BytesIO(r.content))


def log_dummy_data():
    run_names = [
        "adventurous-aardvark-1",
        "bountiful-badger-2",
        "clairvoyant-chipmunk-3",
        "dastardly-duck-4",
        "eloquent-elephant-5",
        "flippant-flamingo-6",
        "giddy-giraffe-7",
        "haughty-hippo-8",
        "ignorant-iguana-9",
        "jolly-jackal-10",
        "kind-koala-11",
        "laughing-lemur-12",
        "manic-mandrill-13",
        "neighbourly-narwhal-14",
        "oblivious-octopus-15",
        "philistine-platypus-16",
        "quant-quail-17",
        "rowdy-rhino-18",
        "solid-snake-19",
        "timid-tarantula-20",
        "understanding-unicorn-21",
        "voracious-vulture-22",
        "wu-tang-23",
        "xenic-xerneas-24",
        "yielding-yveltal-25",
        "zooming-zygarde-26",
    ]

    opts = ["adam", "sgd"]
    encoders = ["resnet18", "resnet50"]
    learning_rates = [0.01]
    for (i, run_name), (opt, encoder, lr) in zip(
        enumerate(run_names), product(opts, encoders, learning_rates)
    ):
        config = {
            "optimizer": opt,
            "encoder": encoder,
            "learning_rate": lr,
            "momentum": 0.1 * random.random(),
        }
        displacement1 = random.random() * 2
        displacement2 = random.random() * 4
        with wandb.init(
            entity=ENTITY, project=PROJECT, config=config, name=run_name
        ) as run:
            for step in range(1000):
                wandb.log(
                    {
                        "acc": 0.1
                        + 0.4
                        * (
                            math.log(1 + step + random.random())
                            + random.random() * run.config.learning_rate
                            + random.random()
                            + displacement1
                            + random.random() * run.config.momentum
                        ),
                        "val_acc": 0.1
                        + 0.4
                        * (
                            math.log(1 + step + random.random())
                            + random.random() * run.config.learning_rate
                            - random.random()
                            + displacement1
                        ),
                        "loss": 0.1
                        + 0.08
                        * (
                            3.5
                            - math.log(1 + step + random.random())
                            + random.random() * run.config.momentum
                            + random.random()
                            + displacement2
                        ),
                        "val_loss": 0.1
                        + 0.04
                        * (
                            4.5
                            - math.log(1 + step + random.random())
                            + random.random() * run.config.learning_rate
                            - random.random()
                            + displacement2
                        ),
                    }
                )

    with wandb.init(
        entity=ENTITY, project=PROJECT, config=config, name=run_names[i + 1]
    ) as run:
        img = get_image(
            "https://www.akc.org/wp-content/uploads/2017/11/Shiba-Inu-standing-in-profile-outdoors.jpg"
        )
        image = wandb.Image(img)
        df = pd.DataFrame(
            {
                "int": [1, 2, 3, 4],
                "float": [1.2, 2.3, 3.4, 4.5],
                "str": ["a", "b", "c", "d"],
                "img": [image] * 4,
            }
        )
        run.log({"img": image, "my-table": df})


class Step:
    def __init__(self, j, r, u, o, at=None):
        self.job_type = j
        self.runs = r
        self.uses_per_run = u
        self.outputs_per_run = o
        self.artifact_type = at if at is not None else "model"
        self.artifacts = []


def create_artifact(name: str, type: str, content: str):
    art = wandb.Artifact(name, type)
    with open("boom.txt", "w") as f:
        f.write(content)
    art.add_file("boom.txt", "test-name")

    img = get_image(
        "https://www.akc.org/wp-content/uploads/2017/11/Shiba-Inu-standing-in-profile-outdoors.jpg"
    )
    image = wandb.Image(img)
    df = pd.DataFrame(
        {
            "int": [1, 2, 3, 4],
            "float": [1.2, 2.3, 3.4, 4.5],
            "str": ["a", "b", "c", "d"],
            "img": [image] * 4,
        }
    )
    art.add(wandb.Table(dataframe=df), "dataframe")
    return art


def log_dummy_lineage():
    pipeline = [
        Step("dataset-generator", 1, 0, 3, "dataset"),
        Step("trainer", 4, (1, 2), 3),
        Step("evaluator", 2, 1, 3),
        Step("ensemble", 1, 1, 1),
    ]
    for (i, step) in enumerate(pipeline):
        for _ in range(step.runs):
            with wandb.init(project=LINEAGE_PROJECT, job_type=step.job_type) as run:
                # use
                uses = step.uses_per_run
                if type(uses) == tuple:
                    uses = random.choice(list(uses))

                if i > 0:
                    prev_step = pipeline[i - 1]
                    input_artifacts = random.sample(prev_step.artifacts, uses)
                    for a in input_artifacts:
                        run.use_artifact(a)
                # log output artifacts
                for j in range(step.outputs_per_run):
                    # name = ''.join(random.choices(string.ascii_lowercase + string.digits, k=6))
                    name = f"{step.artifact_type}-{j}"
                    content = "".join(
                        random.choices(string.ascii_lowercase + string.digits, k=12)
                    )
                    art = create_artifact(name, step.artifact_type, content)
                    run.log_artifact(art)
                    art.wait()

                    # save in pipeline
                    step.artifacts.append(art)

if LOG_DUMMY_RUNS:
  log_dummy_data()
  log_dummy_lineage()


# 🚀 Quickstart! <a id='quickstart'></a>

In [ ]:
import wandb.apis.reports as wr

## Create, save, and load reports
- NOTE: Reports are not saved automatically to reduce clutter.  Explicitly save the report by calling `report.save()`

In [ ]:
report = wr.Report(
    project=PROJECT,
    title='Quickstart Report',
    description="That was easy!"
)                                 # Create
report.save()                     # Save
wr.Report.from_url(report.url)    # Load

## Add content via blocks
- Use blocks to add content like text, images, code, and more
- See `wr.blocks` for all available blocks

In [ ]:
report.blocks = [
    wr.TableOfContents(),
    wr.H1("Text and images example"),
    wr.P("Lorem ipsum dolor sit amet. Aut laborum perspiciatis sit odit omnis aut aliquam voluptatibus ut rerum molestiae sed assumenda nulla ut minus illo sit sunt explicabo? Sed quia architecto est voluptatem magni sit molestiae dolores. Non animi repellendus ea enim internos et iste itaque quo labore mollitia aut omnis totam."),
    wr.Image('https://api.wandb.ai/files/telidavies/images/projects/831572/8ad61fd1.png', caption='Craiyon generated images'),
    wr.P("Et voluptatem galisum quo facilis sequi quo suscipit sunt sed iste iure! Est voluptas adipisci et doloribus commodi ab tempore numquam qui tempora adipisci. Eum sapiente cupiditate ut natus aliquid sit dolor consequatur?"),
]
report.save()

## Add charts and more via Panel Grid
- `PanelGrid` is a special type of block that holds `runsets` and `panels`
  - `runsets` organize data logged to W&B
  - `panels` visualize runset data.  For a full set of panels, see `wr.panels`

In [ ]:
pg = wr.PanelGrid(
    runsets=[
        wr.Runset(ENTITY, PROJECT, "First Run Set"),
        wr.Runset(ENTITY, PROJECT, "Elephants Only!", query="elephant"),
    ],
    panels=[
        wr.LinePlot(x='Step', y=['val_acc'], smoothing_factor=0.8),
        wr.BarPlot(metrics=['acc']),
        wr.MediaBrowser(media_keys='img', num_columns=1),
        wr.RunComparer(diff_only='split', layout={'w': 24, 'h': 9}),
    ]
)

report.blocks = report.blocks[:1] + [wr.H1("Panel Grid Example"), pg] + report.blocks[1:]
report.save()

## Add data lineage with Artifact blocks
- There are equivalent weave panels as well

In [ ]:
artifact_lineage = wr.WeaveBlockArtifact(entity=ENTITY, project=LINEAGE_PROJECT, artifact='model-1', tab='lineage')

report.blocks = report.blocks[:1] + [wr.H1("Artifact lineage example"), artifact_lineage] + report.blocks[1:]
report.save()

## Customize run colors
- Pass in a `dict[run_name, color]`

In [ ]:
pg.custom_run_colors = {
  'adventurous-aardvark-1': '#e84118',
  'bountiful-badger-2':     '#fbc531',
  'clairvoyant-chipmunk-3': '#4cd137',
  'dastardly-duck-4':       '#00a8ff',
  'eloquent-elephant-5':    '#9c88ff',
}
report.save()

## Customize run sets with grouping, filtering, and ordering
- Click on the different run sets in the iframe below and see how they are different
- Grouping: Pass in a list of columns to group by
- Filtering: Use `set_filters_with_python_expr` and pass in a valid python expression.  The syntax is similar to `pandas.DataFrame.query`
- Ordering: Pass in a list of columns where each value is prefixed with `+` for ascending or `-` for descending.

In [ ]:
pg.runsets = [
    wr.Runset(ENTITY, PROJECT, name="Grouping", groupby=["encoder"]),
    wr.Runset(ENTITY, PROJECT, name="Filtering").set_filters_with_python_expr("encoder == 'resnet18' and loss < 0.05"),
    wr.Runset(ENTITY, PROJECT, name="Ordering", order=["+momentum", "-Name"]),
]
report.save()

## Customize group colors
- Pass in a `dict[ordertuple, color]`, where `ordertuple: tuple[runset_name, *groupby_values]`
- For example:
  - Your runset is named `MyRunset`
  - Your runset groupby is `["encoder", "optimizer"]`
  - Then your tuple can be `("MyRunset", "resnet18", "adam")`

In [ ]:
pg.custom_run_colors = {
  ('Grouping', 'resnet50'): 'red',
  ('Grouping', 'resnet18'): 'blue',
  
  # you can do both grouped and ungrouped colors in the same dict
  'adventurous-aardvark-1': '#e84118',
  'bountiful-badger-2':     '#fbc531',
  'clairvoyant-chipmunk-3': '#4cd137',
  'dastardly-duck-4':       '#00a8ff',
  'eloquent-elephant-5':    '#9c88ff',
}
report.save()

# ❓ FAQ <a id='faq'></a>

## My chart is not rendering as expected.
- We try to guess the column type, but sometimes we fail.
- Try prefixing:
  - `c::` for config values
  - `s::` for summary metrics
  - e.g. if your config value was `optimizer`, try `c::optimizer`

## My report is too wide/narrow
- Change the report's width to the right size for you.

In [ ]:
report2 = report.save(clone=True)
report2.width = 'fluid'
report2.save()

## How do I resize panels?
- Pass a `dict[dim, int]` to `panel.layout`
- `dim` is a dimension, which can be `x`, `y` (the coordiantes of the top left corner) `w`, `h` (the size of the panel)
- You can pass any or all dimensions at once
- The space between two dots in a panel grid is 2.

In [ ]:
report = wr.Report(
    PROJECT,
    title="Resizing panels",
    description="Look at this wide parallel coordinates plot!",
    blocks=[
        wr.PanelGrid(
            panels=[
                wr.ParallelCoordinatesPlot(
                    columns=[
                        wr.PCColumn("Step"),
                        wr.PCColumn("c::model"),
                        wr.PCColumn("c::optimizer"),
                        wr.PCColumn("Step"),
                        wr.PCColumn("val_acc"),
                        wr.PCColumn("val_loss"),
                    ],
                    layout={'w': 24, 'h': 9}  # change the layout!
                ),
            ]
        )
    ]
)
report.save()

## What blocks are available?
- See `wr.blocks` for a list of blocks.
- In an IDE or notebook, you can also do `wr.blocks.<tab>` to get autocomplete.

In [ ]:
# Panel grid is omitted.  See next section for PanelGrid and panels
report = wr.Report(
    PROJECT,
    title='W&B Block Gallery',
    description="Check out all of the blocks available in W&B",
    blocks=[
        wr.H1(text="Heading 1"),
        wr.P(text="Normal paragraph"),
        wr.H2(text="Heading 2"),
        wr.P(
            [
                "here is some text, followed by",
                wr.InlineCode("select * from code in line"),
                "and then latex",
                wr.InlineLaTeX("e=mc^2"),
            ]
        ),
        wr.H3(text="Heading 3"),
        wr.CodeBlock(
            code=["this:", "- is", "- a", "cool:", "- yaml", "- file"],
            language="yaml",
        ),
        wr.WeaveBlockSummaryTable(ENTITY, PROJECT, 'my-table'),
        wr.WeaveBlockArtifact(ENTITY, LINEAGE_PROJECT, 'model-1', 'lineage'),
        wr.WeaveBlockArtifactVersionedFile(ENTITY, LINEAGE_PROJECT, 'model-1', 'v0', "dataframe.table.json"),
        wr.MarkdownBlock(text="Markdown cell with *italics* and **bold** and $e=mc^2$"),
        wr.LaTeXBlock(text="\\gamma^2+\\theta^2=\\omega^2\n\\\\ a^2 + b^2 = c^2"),
        wr.Image("https://api.wandb.ai/files/megatruong/images/projects/918598/350382db.gif", caption="It's a me, Pikachu"),
        wr.UnorderedList(items=["Bullet 1", "Bullet 2"]),
        wr.OrderedList(items=["Ordered 1", "Ordered 2"]),
        wr.CheckedList(items=["Unchecked", "Checked"], checked=[False, True]),
        wr.BlockQuote(text="Block Quote 1\nBlock Quote 2\nBlock Quote 3"),
        wr.CalloutBlock(text=["Callout 1", "Callout 2", "Callout 3"]),
        wr.HorizontalRule(),
        wr.Video(url="https://www.youtube.com/embed/6riDJMI-Y8U"),
        wr.Spotify(spotify_id="5cfUlsdrdUE4dLMK7R9CFd"),
        wr.SoundCloud(url="https://api.soundcloud.com/tracks/1076901103"),
    ]
).save()
report.blocks += [wr.Gallery(ids=[report.id])]  # get report id on save
report.save()

## What panels are available?
- See `wr.panels` for a list of panels
- In an IDE or notebook, you can also do `wr.panels.<tab>` to get autocomplete.
- Panels have a lot of settings.  Inspect the panel to see what you can do!

In [ ]:
report = wr.Report(
    project=PROJECT,
    title='W&B Panel Gallery',
    description="Check out all of the panels available in W&B",
    width='fluid',
    blocks=[
        wr.PanelGrid(
            runsets=[
                wr.Runset(project=LINEAGE_PROJECT),
                wr.Runset(),
            ],
            panels=[
                wr.MediaBrowser(media_keys="img"),
                wr.MarkdownPanel("Hello *italic* **bold** $e=mc^2$ `something`"),
                
                # LinePlot showed with many settings enabled for example
                wr.LinePlot(
                    title="Validation Accuracy over Time",
                    x="Step",
                    y=["val_acc"],
                    range_x=[0, 1000],
                    range_y=[1, 4],
                    log_x=True,
                    title_x="Training steps",
                    title_y="Validation Accuracy",
                    ignore_outliers=True,
                    groupby='encoder',
                    groupby_aggfunc="mean",
                    groupby_rangefunc="minmax",
                    smoothing_factor=0.5,
                    smoothing_type="gaussian",
                    smoothing_show_original=True,
                    max_runs_to_show=10,
                    font_size="large",
                    legend_position="west",
                ),
                wr.ScatterPlot(
                    title="Validation Accuracy vs. Validation Loss",
                    x="val_acc",
                    y="val_loss",
                    log_x=False,
                    log_y=False,
                    running_ymin=True,
                    running_ymean=True,
                    running_ymax=True,
                    font_size="small",
                    regression=True,
                ),
                wr.BarPlot(
                    title="Validation Loss by Encoder",
                    metrics=["val_loss"],
                    orientation='h',
                    range_x=[0, 0.11],
                    title_x="Validation Loss",
                    # title_y="y axis title",
                    groupby='encoder',
                    groupby_aggfunc="median",
                    groupby_rangefunc="stddev",
                    max_runs_to_show=20,
                    max_bars_to_show=3,
                    font_size="auto",
                ),
                wr.ScalarChart(
                    title="Maximum Number of Steps",
                    metric="Step",
                    groupby_aggfunc="max",
                    groupby_rangefunc="stderr",
                    font_size="large",
                ),
                wr.CodeComparer(diff="split"),
                wr.ParallelCoordinatesPlot(
                    columns=[
                        wr.PCColumn("Step"),
                        wr.PCColumn("c::model"),
                        wr.PCColumn("c::optimizer"),
                        wr.PCColumn("Step"),
                        wr.PCColumn("val_acc"),
                        wr.PCColumn("val_loss"),
                    ],
                ),
                wr.ParameterImportancePlot(with_respect_to="val_loss"),
                wr.RunComparer(diff_only="split"),
                wr.CustomChart(
                    query={'summary': ['val_loss', 'val_acc']},
                    chart_name='wandb/scatter/v0',
                    chart_fields={'x': 'val_loss', 'y': 'val_acc'}
                ),
                wr.WeavePanelSummaryTable("my-table"),
                wr.WeavePanelArtifact('model-1', 'lineage', layout={'w': 24, 'h': 12}),
                wr.WeavePanelArtifactVersionedFile('model-1', 'v0', "dataframe.table.json", layout={'w': 24, 'h': 12}),
            ],
        ),
    ]
)
report.save()

## What above weave?
- A limited subset of weave panels are available today, including Artifact, ArtifactVersionedFile, and SummaryTable
- Stay tuned for future updates around weave support!

## Can I use this in CI (e.g. create a report for each git commit?)
- Yep!  Check out [this example](https://github.com/andrewtruong/wandb-gh-actions/actions/runs/3476558992) which creates a report via Github Actions

## How can I link related reports together?
- Suppose have have two reports like below:

In [ ]:
report1 = wr.Report(
    PROJECT,
    title='Report 1',
    description="Great content coming from Report 1",
    blocks=[
        wr.H1('Heading from Report 1'),
        wr.P('Lorem ipsum dolor sit amet. Aut fuga minus nam vero saepeA aperiam eum omnis dolorum et ducimus tempore aut illum quis aut alias vero. Sed explicabo illum est eius quianon vitae sed voluptatem incidunt. Vel architecto assumenda Ad voluptatem quo dicta provident et velit officia. Aut galisum inventoreSed dolore a illum adipisci a aliquam quidem sit corporis quia cum magnam similique.'),
        wr.PanelGrid(
            panels=[
                wr.LinePlot(x='global_step', y=['charts/episodic_return'], smoothing_factor=0.85, groupby_aggfunc='mean', groupby_rangefunc='minmax', layout={'x': 0, 'y': 0, 'w': 12, 'h': 8}),
                wr.MediaBrowser(media_keys="videos", num_columns=4, layout={'w': 12, 'h': 8}),
            ],
            runsets=[
                wr.Runset(entity='openrlbenchmark', project='cleanrl', query='bigfish', groupby=['env_id', 'exp_name'])
            ],
            custom_run_colors={
                ('Run set', 'bigfish', 'ppg_procgen'): "#2980b9",
                ('Run set', 'bigfish', 'ppo_procgen'): "#e74c3c",
            }
        ),
    ]
).save()

report2 = wr.Report(
    PROJECT,
    title='Report 2',
    description="Great content coming from Report 2",
    blocks=[
        wr.H1('Heading from Report 2'),
        wr.P('Est quod ducimus ut distinctio corruptiid optio qui cupiditate quibusdam ea corporis modi. Eum architecto vero sed error dignissimosEa repudiandae a recusandae sint ut sint molestiae ea pariatur quae. In pariatur voluptas ad facere neque 33 suscipit et odit nostrum ut internos molestiae est modi enim. Et rerum inventoreAut internos et dolores delectus aut Quis sunt sed nostrum magnam ab dolores dicta.'),
        wr.PanelGrid(
            panels=[
                wr.LinePlot(x='global_step', y=['charts/SPS']),
                wr.LinePlot(x='global_step', y=['charts/episodic_length']),
                wr.LinePlot(x='global_step', y=['charts/episodic_return']),
            ],
            runsets=[
                wr.Runset("openrlbenchmark", "cleanrl", "DQN", groupby=["exp_name"]).set_filters_with_python_expr("env_id == 'BreakoutNoFrameskip-v4' and exp_name == 'dqn_atari'"),
                wr.Runset("openrlbenchmark", "cleanrl", "SAC-discrete 0.8", groupby=["exp_name"]).set_filters_with_python_expr("env_id == 'BreakoutNoFrameskip-v4' and exp_name == 'sac_atari' and target_entropy_scale == 0.8"),
                wr.Runset("openrlbenchmark", "cleanrl", "SAC-discrete 0.88", groupby=["exp_name"]).set_filters_with_python_expr("env_id == 'BreakoutNoFrameskip-v4' and exp_name == 'sac_atari' and target_entropy_scale == 0.88"),
            ],
            custom_run_colors={
                ('DQN',               'dqn_atari'): '#e84118',
                ('SAC-discrete 0.8',  'sac_atari'): '#fbc531',
                ('SAC-discrete 0.88', 'sac_atari'): '#00a8ff',
            }
        ),
    ]
).save()

### Combine blocks into a new report

In [ ]:
report = wr.Report(PROJECT,
    title="Report with links",
    description="Use `wr.Link(text, url)` to add links inside normal text, or use normal markdown syntax in a MarkdownBlock",
    blocks=[
        wr.H1("This is a normal heading"),
        wr.P("And here is some normal text"),

        wr.H1(["This is a heading ", wr.Link("with a link!", url="https://wandb.ai/")]),
        wr.P(["Most text formats support ", wr.Link("adding links", url="https://wandb.ai/")]),

        wr.MarkdownBlock("""You can also use markdown syntax for [links](https://wandb.ai/)""")
    ]
)
report.save()

In [ ]:
report3 = wr.Report(
    PROJECT,
    title="Combined blocks report",
    description="This report combines blocks from both Report 1 and Report 2",
    blocks=[*report1.blocks, *report2.blocks]
)
report3.save()

### Reference the two reports in a gallery block

In [ ]:
report4 = wr.Report(
    PROJECT,
    title="Referenced reports via Gallery",
    description="This report has gallery links to Report1 and Report 2",
    blocks=[wr.Gallery(ids=[report1.id, report2.id])]
)
report4.save()

## How do I add links to text?
- Use `wr.Link`

## Do you support markdown?
Yep
- In blocks, use `wr.MarkdownBlock`
- In panels, use `wr.MarkdownPanel`

In [ ]:
markdown = """
# Ducat quicquam

## Egit sit utque torta cuncta si oret

Lorem markdownum voco sacrorum religata. Erat ante condita mellaque Cimmerios
cultoribus pictas manu usquam illa umbra potentia inpedienda Padumque [euntes
motae](http://nec.com/necsollemnia.aspx), detraxit! Accessus discrimen,
Cyllenius *e* solum saepe terras perfringit amorem tenent consulit falce
referemus tantum. Illo qui attonitas, Adonis ultra stabunt horret requiescere
quae deam similis miserum consuetas: tantos aegram, metuam. Tetigere
**invidiae** preces indicere populo semper, limine sui dumque, lustra
alimentaque vidi nec corpusque aquarum habebat, in.


## Aurea simile iunctoque dux semper verbis

Vinctorum vidisset et caede officio, viae alia ratione aer regalia, etiamnum.
Occupat tanta; vicem, Ithaceque, et ille nec exclamat. Honori carpserat visae
magniloquo perluitur corpora tamen. Caput an Minervae sed vela est cecidere
luctus umbras iunctisque referat. Accensis **aderis capillos** pendebant
[retentas parvum](http://ipse.com/).

    if (desktop(2)) {
        laser_qwerty_optical.webcam += upsRoom + window;
    }
    if (type) {
        memoryGrayscale(backbone, mask_multimedia_html);
    }
    if (natInterfaceFile == 23 + 92) {
        interface_sku.platform = compressionMotherboard - error_veronica_ata;
        dsl_windows = 57 * -2;
        definition *= -4;
    } else {
        frame(4, market_chip_irq, megapixel_eide);
    }
    if (mashupApiFlash(-1, margin) - graphicSoftwareNas.ddr_samba_reimage(port)
            != control_navigation_pseudocode(yahoo.microcomputerDimm(
            mips_adsl))) {
        postscriptViralDirectx(1, cron_router_voip(669103, managementPitch,
                ospf_up_paper), frame);
        servlet_cross_paper.controlLanguage(insertion_source.viewHorizontalRead(
                enterprise, widget, parse_encoding), end);
        script_e(rateRss(yobibyte, fddi, vci_hyper_joystick), surgeHeat / case);
    }
    pcmciaRealSystem.basic_exbibyte_controller = carrier.domainDesktop(-4 +
            laptop + 5);
"""

report = wr.Report(
    PROJECT,
    title="Report with markdown",
    description="See what's possible with MarkdownBlock and MarkdownPanel",
    blocks=[
        wr.MarkdownBlock(markdown),
        wr.PanelGrid(
            panels=[
                wr.MarkdownPanel(markdown,                layout={'w': 12, 'h': 18}),
                wr.LinePlot(x='Step', y=['val_acc'] ,     layout={'x': 12, 'y': 0}),
                wr.LinePlot(x='Step', y=['val_loss'],     layout={'x': 12, 'y': 6}),
                wr.ScatterPlot(x='val_loss', y='val_acc', layout={'x': 12, 'y': 12}),
            ]
        )
    ]
)
report.save()

## Can I build the report up from smaller pieces / all at once?
Yep.  We'll demonstrate by putting together a report with a parallel coordinates plot.

NOTE: this section assumes you have run the [sweeps notebook](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Organizing_Hyperparameter_Sweeps_in_PyTorch_with_W%26B.ipynb) already.

### Build it up incrementally
As you might do if you were creating a report in the UI

1. Create a report

In [ ]:
report = wr.Report(project=PROJECT, title='Parallel Coordinates Example', description="Using the pytorch sweeps demo")

2. Add a panel grid

In [ ]:
pg = wr.PanelGrid()
report.blocks = [pg]

3. Specify your runsets

In [ ]:
pg.runsets = [wr.Runset(project='pytorch-sweeps-demo')]

4. Specify your panels

In [ ]:
pg.panels = [
    wr.ParallelCoordinatesPlot(
        columns=[
            wr.PCColumn(metric="c::batch_size"),
            wr.PCColumn(metric="c::dropout"),
            wr.PCColumn(metric="c::epochs"),
            wr.PCColumn(metric="c::fc_layer_size"),
            wr.PCColumn(metric="c::learning_rate"),
            wr.PCColumn(metric="c::optimizer"),
            wr.PCColumn(metric="loss"),
        ]
    )
]


5. Save the report

In [ ]:
report.save()

### The same thing all-in-one

In [ ]:
report = wr.Report(
    project=PROJECT,
    title="Parallel Coordinates Example (all-in-one)",
    description="Using the pytorch sweeps demo (same as the other one but written in one expression)",
    blocks=[
        wr.PanelGrid(
            runsets=[wr.Runset(project="pytorch-sweeps-demo")],
            panels=[
                wr.ParallelCoordinatesPlot(
                    columns=[
                        wr.PCColumn(metric='c::batch_size'),
                        wr.PCColumn(metric='c::dropout'),
                        wr.PCColumn(metric='c::epochs'),
                        wr.PCColumn(metric='c::fc_layer_size'),
                        wr.PCColumn(metric='c::learning_rate'),
                        wr.PCColumn(metric='c::optimizer'),
                        wr.PCColumn(metric='loss'),
                    ]
                )
            ],
        )
    ],
)

In [ ]:
report.save()

## I tried mutating an object in list but it didn't work!
tl;dr: It should always work if you assign a value to the attribute instead of mutating.  If you really need to mutate, do it before assignment.

---

This can happen in a few places that contain lists of wandb objects, e.g.:
- `report.blocks`
- `panel_grid.panels`
- `panel_grid.runsets`

In [ ]:
report = wr.Report(project=PROJECT)

Good: Assign `b`

In [ ]:
b = wr.H1(text=["Hello", " World!"])
report.blocks = [b]
assert b.text == ["Hello", " World!"]
assert report.blocks[0].text == ["Hello", " World!"]

Bad: Mutate `b` without reassigning

In [ ]:
b.text = ["Something", " New"]
assert b.text == ["Something", " New"]
assert report.blocks[0].text == ["Hello", " World!"]

Good: Mutate `b` and then reassign it

In [ ]:
report.blocks = [b]
assert b.text == ["Something", " New"]
assert report.blocks[0].text == ["Something", " New"]

## How do I show tables?

In [ ]:
report = wr.Report(project=PROJECT, title='Adding tables to reports', description="Add tables with WeaveBlockSummaryTable or WeavePanelSummaryTable")

### Using weave blocks

In [ ]:
report.blocks += [wr.WeaveBlockSummaryTable(ENTITY, PROJECT, "my-table")]
report.save()

### Using weave panels (via PanelGrid)

In [ ]:
report.blocks += [
    wr.PanelGrid(
        panels=[wr.WeavePanelSummaryTable("my-table")]
    )
]
report.save()

## How do I show artifact lineage / versions?

In [ ]:
report = wr.Report(project=PROJECT, title='Adding artifact lineage to reports', description="via WeaveBlockArtifact, WeaveBlockArtifactVersionedFile, or their panel equivalents")

### Using weave blocks

In [ ]:
report.blocks += [
    wr.WeaveBlockArtifact(ENTITY, LINEAGE_PROJECT, "model-1", "lineage"),
    wr.WeaveBlockArtifactVersionedFile(ENTITY, LINEAGE_PROJECT, "model-1", "v0", "dataframe.table.json")
]
report.save()

### Using weave panels (via PanelGrid)

In [ ]:
report.blocks += [
    wr.PanelGrid(panels=[
        wr.WeavePanelArtifact("", "lineage"),
        wr.WeavePanelArtifactVersionedFile("", "v0", "dataframe.table.json")
    ])
]
report.save()

## How can I create report templates?
- See some of the examples in `wr.templates`
- The most straightforward way is to create a function that returns your target report and/or its blocks.

### A basic template
- Just use a function

In [ ]:
def my_report_template(title, description, project, metric):
    return wr.Report(
        title=title,
        description=description,
        project=project,
        blocks=[
            wr.H1(f"Look at our amazing metric called `{metric}`"),
            wr.PanelGrid(
                panels=[wr.LinePlot(x='Step', y=metric, layout={'w': 24, 'h': 8})],
            )
        ]
    ).save()

In [ ]:
my_report_template('My templated report', "Here's an example of how you can make a function for templates", PROJECT, 'val_acc')

### More advanced templates

In [ ]:
def create_header():
  return [
    wr.P(),
    wr.HorizontalRule(),
    wr.P(),
    wr.Image(
      "https://camo.githubusercontent.com/83839f20c90facc062330f8fee5a7ab910fdd04b80b4c4c7e89d6d8137543540/68747470733a2f2f692e696d6775722e636f6d2f676236423469672e706e67"
    ),
    wr.P(),
    wr.HorizontalRule(),
    wr.P(),
  ]

def create_footer():
  return [
    wr.P(),
    wr.HorizontalRule(),
    wr.P(),
    wr.H1("Disclaimer"),
    wr.P(
      "The views and opinions expressed in this report are those of the authors and do not necessarily reflect the official policy or position of Weights & Biases. blah blah blah blah blah boring text at the bottom"
    ),
    wr.P(),
    wr.HorizontalRule(),
  ]

def create_main_content(metric):
  return [
    wr.H1(f"Look at our amazing metric called `{metric}`"),
    wr.PanelGrid(
      panels=[wr.LinePlot(x='Step', y=metric, layout={'w': 24, 'h': 8})],
    )
  ]

def create_templated_report_with_header_and_footer(title, project, metric):
  return wr.Report(
    title=title,
    project=project,
    blocks=[
      *create_header(),
      *create_main_content(metric),
      *create_footer(),
    ]).save()

In [ ]:
create_templated_report_with_header_and_footer(title="Another templated report", project=PROJECT, metric='val_acc')

# 📌 Complete Examples <a id='complete_examples'></a>

## Reinforcement Learning (RL)

In [ ]:
wr.Report(
    project=PROJECT,
    title='Reinforcement Learning Report',
    description='Aut totam dolores aut galisum atque aut placeat quia. Vel quisquam omnis ut quibusdam doloremque a delectus quia in omnis deserunt. Quo ipsum beatae aut veniam earum non ipsa reiciendis et fugiat asperiores est veritatis magni et corrupti internos. Ut quis libero ut alias reiciendis et animi delectus.',
    blocks=[
        wr.TableOfContents(),
        wr.H1("Ea quidem illo est dolorem illo."),
        wr.P("Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed ac eros ut nunc venenatis tincidunt vel ut dolor. Sed sed felis dictum, congue risus vel, aliquet dolor. Donec ut risus vel leo dictum tristique. Nunc sed urna mi. Morbi nulla turpis, vehicula eu maximus ut, gravida id libero. Duis porta risus leo, quis lobortis enim ultrices a. Donec quam augue, vestibulum vitae mollis at, tincidunt non orci. Morbi faucibus dignissim tempor. Vestibulum ornare augue a orci tincidunt porta. Pellentesque et ante et purus gravida euismod. Maecenas sit amet sollicitudin felis, sed egestas nunc."),
        wr.H2('Et sunt sunt eum asperiores ratione.'),
        wr.PanelGrid(
            panels=[
                wr.LinePlot(x='global_step', y=['charts/episodic_return'], smoothing_factor=0.85, groupby_aggfunc='mean', groupby_rangefunc='minmax', layout={'x': 0, 'y': 0, 'w': 12, 'h': 8}),
                wr.MediaBrowser(media_keys="videos", num_columns=4, layout={'w': 12, 'h': 8}),
            ],
            runsets=[
                wr.Runset(entity='openrlbenchmark', project='cleanrl', query='bigfish', groupby=['env_id', 'exp_name'])
            ],
            custom_run_colors={
                ('Run set', 'bigfish', 'ppg_procgen'): "#2980b9",
                ('Run set', 'bigfish', 'ppo_procgen'): "#e74c3c",
            }
        ),
        wr.H2('Sit officia inventore non omnis deleniti.'),
        wr.PanelGrid(
            panels=[
                wr.LinePlot(x='global_step', y=['charts/episodic_return'], smoothing_factor=0.85, groupby_aggfunc='mean', groupby_rangefunc='minmax', layout={'x': 0, 'y': 0, 'w': 12, 'h': 8}),
                wr.MediaBrowser(media_keys="videos", num_columns=4, layout={'w': 12, 'h': 8}),
            ],
            runsets=[
                wr.Runset(entity='openrlbenchmark', project='cleanrl', query='starpilot', groupby=['env_id', 'exp_name'])
            ],
            custom_run_colors={
                ('Run set', 'starpilot', 'ppg_procgen'): "#2980b9",
                ('Run set', 'starpilot', 'ppo_procgen'): "#e74c3c",
            }
        ),
        wr.H2('Aut amet nesciunt vel quisquam repellendus sed labore voluptas.'),
        wr.PanelGrid(
            panels=[
                wr.LinePlot(x='global_step', y=['charts/episodic_return'], smoothing_factor=0.85, groupby_aggfunc='mean', groupby_rangefunc='minmax', layout={'x': 0, 'y': 0, 'w': 12, 'h': 8}),
                wr.MediaBrowser(media_keys="videos", num_columns=4, layout={'x': 0, 'y': 0, 'w': 12, 'h': 8}),
            ],
            runsets=[
                wr.Runset(entity='openrlbenchmark', project='cleanrl', query='bossfight', groupby=['env_id', 'exp_name'])
            ],
            custom_run_colors={
                ('Run set', 'bossfight', 'ppg_procgen'): "#2980b9",
                ('Run set', 'bossfight', 'ppo_procgen'): "#e74c3c",
            }
        ),
        wr.HorizontalRule(),
        wr.H1("Sed consectetur vero et voluptas voluptatem et adipisci blanditiis."),
        wr.P("Sit aliquid repellendus et numquam provident quo quaerat earum 33 sunt illo et quos voluptate est officia deleniti. Vel architecto nulla ex nulla voluptatibus qui saepe officiis quo illo excepturi ea dolorum reprehenderit."),
        wr.H2("Qui debitis iure 33 voluptatum eligendi."),
        wr.P("Non veniam laudantium et fugit distinctio qui aliquid eius sed laudantium consequatur et quia perspiciatis. Et odio inventore est voluptas fugiat id perspiciatis dolorum et perferendis recusandae vel Quis odio 33 beatae veritatis. Ex sunt accusamus aut soluta eligendi sed perspiciatis maxime 33 dolorem dolorum est aperiam minima. Et earum rerum eos illo sint eos temporibus similique ea fuga iste sed quia soluta sit doloribus corporis sed tenetur excepturi?"),
        wr.PanelGrid(
            panels=[
                wr.LinePlot(x='global_step', y=['charts/SPS']),
                wr.LinePlot(x='global_step', y=['charts/episodic_length']),
                wr.LinePlot(x='global_step', y=['charts/episodic_return']),
            ],
            runsets=[
                wr.Runset("openrlbenchmark", "cleanrl", "DQN", groupby=["exp_name"]).set_filters_with_python_expr("env_id == 'BreakoutNoFrameskip-v4' and exp_name == 'dqn_atari'"),
                wr.Runset("openrlbenchmark", "cleanrl", "SAC-discrete 0.8", groupby=["exp_name"]).set_filters_with_python_expr("env_id == 'BreakoutNoFrameskip-v4' and exp_name == 'sac_atari' and target_entropy_scale == 0.8"),
                wr.Runset("openrlbenchmark", "cleanrl", "SAC-discrete 0.88", groupby=["exp_name"]).set_filters_with_python_expr("env_id == 'BreakoutNoFrameskip-v4' and exp_name == 'sac_atari' and target_entropy_scale == 0.88"),
            ],
            custom_run_colors={
                ('DQN',               'dqn_atari'): '#e84118',
                ('SAC-discrete 0.8',  'sac_atari'): '#fbc531',
                ('SAC-discrete 0.88', 'sac_atari'): '#00a8ff',
            }
        ),
    ]
).save()

## Customer Landing Page

In [ ]:
report = wr.templates.create_customer_landing_page(
    project=PROJECT,
    company_name='Company',
    main_contact='Contact McContact (email@company.com)',
    slack_link='https://company.slack.com/blah',
)
report.save()

## Enterprise Report with Branded Header and Footer

In [ ]:
report = wr.templates.create_enterprise_report(
    project=PROJECT,
    body=[
        wr.H1("Ea quidem illo est dolorem illo."),
        wr.P("Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed ac eros ut nunc venenatis tincidunt vel ut dolor. Sed sed felis dictum, congue risus vel, aliquet dolor. Donec ut risus vel leo dictum tristique. Nunc sed urna mi. Morbi nulla turpis, vehicula eu maximus ut, gravida id libero. Duis porta risus leo, quis lobortis enim ultrices a. Donec quam augue, vestibulum vitae mollis at, tincidunt non orci. Morbi faucibus dignissim tempor. Vestibulum ornare augue a orci tincidunt porta. Pellentesque et ante et purus gravida euismod. Maecenas sit amet sollicitudin felis, sed egestas nunc."),
        wr.H2('Et sunt sunt eum asperiores ratione.'),
        wr.PanelGrid(
            panels=[
                wr.LinePlot(x='global_step', y=['charts/episodic_return'], smoothing_factor=0.85, groupby_aggfunc='mean', groupby_rangefunc='minmax', layout={'x': 0, 'y': 0, 'w': 12, 'h': 8}),
                wr.MediaBrowser(media_keys="videos", num_columns=4, layout={'w': 12, 'h': 8}),
            ],
            runsets=[
                wr.Runset(entity='openrlbenchmark', project='cleanrl', query='bigfish', groupby=['env_id', 'exp_name'])
            ],
            custom_run_colors={
                ('Run set', 'bigfish', 'ppg_procgen'): "#2980b9",
                ('Run set', 'bigfish', 'ppo_procgen'): "#e74c3c",
            }
        ),
        wr.H2('Sit officia inventore non omnis deleniti.'),
        wr.PanelGrid(
            panels=[
                wr.LinePlot(x='global_step', y=['charts/episodic_return'], smoothing_factor=0.85, groupby_aggfunc='mean', groupby_rangefunc='minmax', layout={'x': 0, 'y': 0, 'w': 12, 'h': 8}),
                wr.MediaBrowser(media_keys="videos", num_columns=4, layout={'w': 12, 'h': 8}),
            ],
            runsets=[
                wr.Runset(entity='openrlbenchmark', project='cleanrl', query='starpilot', groupby=['env_id', 'exp_name'])
            ],
            custom_run_colors={
                ('Run set', 'starpilot', 'ppg_procgen'): "#2980b9",
                ('Run set', 'starpilot', 'ppo_procgen'): "#e74c3c",
            }
        ),
    ]
)
report.save()